# Full Connected Feed
# 全链接神经网络

In [1]:
import tensorflow as tf
import input_data
import numpy as np
import time
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
np.set_printoptions(suppress=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# Network Parameters
n_input = 784
n_hidden_1 = 256
n_hidden_2 = 256
n_classes = 10
train_dropout = 0.9
test_dropout = 1.0

# Training Parameters
learning_rate = 0.001
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
print(total_batch)
training_epochs = 30

# tf Graph input
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

550


In [3]:
# Create model
def NN(x, weights, biases, dropout):

    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, dropout)

    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_2 = tf.nn.dropout(layer_2, dropout)

    out = tf.add(tf.matmul(layer_2, weights['out']), biases['out'])
    return out

In [4]:
# Construct model
y_pred = NN(X, weights, biases, keep_prob)
y_softmax = tf.nn.softmax(y_pred)

# Define loss and optimizer
# type 1:
# loss = tf.reduce_mean(
#     -tf.reduce_sum(Y * tf.log(y_softmax), reduction_indices=[1]))
# type 2:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=y_pred))
# type 3:
# loss = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# Evaluate model
pred = tf.argmax(y_pred, 1)
true = tf.argmax(Y, 1)
correct_prediction = tf.equal(pred, true)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [5]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Training cycle
all_x_test = mnist.test.images
all_y_test = mnist.test.labels
start = time.time()
for epoch_i in range(training_epochs):
    ave_cost = 0
    for batch_i in range(total_batch):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        _, c = sess.run(
            [optimizer, loss],
            feed_dict={
                X: batch_x,
                Y: batch_y,
                keep_prob: train_dropout
            })
        ave_cost += c / total_batch
    # Display logs per epoch step
    if epoch_i % 1 == 0:
        train_acc = sess.run(
            accuracy,
            feed_dict={
                X: batch_x,
                Y: batch_y,
                keep_prob: test_dropout
            })
        test_acc = sess.run(
            accuracy,
            feed_dict={
                X: all_x_test,
                Y: all_y_test,
                keep_prob: test_dropout
            })
        print("Epoch:%2d Batch:%4d" % (epoch_i + 1, batch_i + 1),
              "train_acc=%.3f" % train_acc, "test_acc=%.3f" % test_acc,
              "train_cost=%5.3f" % ave_cost)
end = time.time()
print("Process Time :%.2f s" % (end - start))

# Calculate accuracy
acc = sess.run(
    accuracy,
    feed_dict={
        X: all_x_test,
        Y: all_y_test,
        keep_prob: test_dropout
    })
print("test accuracy=%.3f" % acc)
sess.close()

Epoch: 1 Batch: 550 train_acc=0.860 test_acc=0.868 train_cost=262.059
Epoch: 2 Batch: 550 train_acc=0.890 test_acc=0.903 train_cost=73.148
Epoch: 3 Batch: 550 train_acc=0.950 test_acc=0.915 train_cost=45.090
Epoch: 4 Batch: 550 train_acc=0.950 test_acc=0.923 train_cost=32.821
Epoch: 5 Batch: 550 train_acc=0.940 test_acc=0.928 train_cost=25.269
Epoch: 6 Batch: 550 train_acc=0.960 test_acc=0.935 train_cost=19.792
Epoch: 7 Batch: 550 train_acc=0.940 test_acc=0.936 train_cost=16.378
Epoch: 8 Batch: 550 train_acc=0.930 test_acc=0.940 train_cost=13.255
Epoch: 9 Batch: 550 train_acc=0.920 test_acc=0.940 train_cost=11.279
Epoch:10 Batch: 550 train_acc=0.980 test_acc=0.945 train_cost=9.282
Epoch:11 Batch: 550 train_acc=0.980 test_acc=0.945 train_cost=7.844
Epoch:12 Batch: 550 train_acc=0.970 test_acc=0.947 train_cost=6.957
Epoch:13 Batch: 550 train_acc=0.960 test_acc=0.950 train_cost=5.939
Epoch:14 Batch: 550 train_acc=0.980 test_acc=0.952 train_cost=5.232
Epoch:15 Batch: 550 train_acc=0.970 te